In [ ]:
lam = 0.01
TX = np.array([[0, 0], [0.005, 0], [0.01, 0], [0.015, 0]])
RX_2 = list(np.array([-10, -40]) + 0.005*np.array([np.cos(math.radians(150)), np.sin(math.radians(150))]))
RX = np.array([[-10, -40], RX_2])
geo = geometry(TX, RX, lam, 0.005, scatterer = np.array([0, 0]), random_phase = math.pi/6)
BT = np.zeros((4, 2), dtype=np.complex128)
BR = np.zeros((2, 2), dtype=np.complex128)
A = np.zeros((2, 2), dtype=np.complex128)
BT[:, 0] = geo.calculate_bT().reshape(-1, )
BR[:, 0] = geo.calculate_bR().reshape(-1, )
A[0, 0] = geo.calculate_a()
geo = geometry(TX, RX, lam, 0.005, scatterer = np.array([20, -25]), random_phase = math.pi/6)
BT[:, 1] = geo.calculate_bT().reshape(-1, )
BR[:, 1] = geo.calculate_bR().reshape(-1, )
A[1, 1] = geo.calculate_a()


H = BR@A@BT.conj().T
H_partial = BR@np.absolute(A)@BT.conj().T

He = BR.conj().T@H_partial@BT
omega = scipy.linalg.inv(scipy.linalg.sqrtm((BR.conj().T)@BR))
ff = scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT))
He2 = omega@He@ff

He_p = BR.conj().T@H@BT
omega = scipy.linalg.inv(scipy.linalg.sqrtm((BR.conj().T)@BR))
ff = scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT))
He3 = omega@He_p@ff
# He2 = BR.conj().T@H@BT



rates1_CSIT_nobeam = []
rates1_pCSIT_nobeam = []
rates1_CSIT_beam = []
rates1_pCSIT_beam = []
for p in np.arange(0, 65, 5):

  #CSIT_nobeam
  PT = 10**(p/10)
  L = 1
  chan_mat = np.zeros((L, L), dtype = object)
  chan_mat[0, 0] = H
  w = list(np.ones(L, dtype = int))
  ss = init_sigma(L, 4, PT)
  _, r, _, sigma, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
  rates1_CSIT_nobeam.append(r)

  #pCSIT_nobeam
  PT = 10**(p/10)
  L = 1
  chan_mat = np.zeros((L, L), dtype = object)
  chan_mat[0, 0] = H_partial
  w = list(np.ones(L, dtype = int))
  ss = init_sigma(L, 4, PT)
  _, _, _, sigma, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
  r = np.log2(np.linalg.det(np.identity(2)+H@sigma[0]@(H.conj().T)))
  rates1_pCSIT_nobeam.append(r)

  #pCSIT_beam
  PT = 10**(p/10)
  L = 1
  chan_mat = np.zeros((L, L), dtype = object)
  chan_mat[0, 0] = He2
  w = list(np.ones(L, dtype = int))
  ss = init_sigma(L, 2, PT)
  _, _, _, sigma, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
  ss = (scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT)))@(sigma[0])@(scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT)))
  Sig = BT@ss@BT.conj().T
  r = np.log2(np.linalg.det(np.identity(2)+H@Sig@(H.conj().T)))
  rates1_pCSIT_beam.append(r)

  #CSIT_beam
  PT = 10**(p/10)
  L = 1
  chan_mat = np.zeros((L, L), dtype = object)
  chan_mat[0, 0] = He3
  w = list(np.ones(L, dtype = int))
  ss = init_sigma(L, 2, PT)
  _, _, _, sigma, _ = alg_dual_link(PT, w, chan_mat, epsilon, Sigma = ss)
  ss = (scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT)))@(sigma[0])@(scipy.linalg.inv(scipy.linalg.sqrtm((BT.conj().T)@BT)))
  Sig = BT@ss@BT.conj().T
  r = np.log2(np.linalg.det(np.identity(2)+H@Sig@(H.conj().T)))
  rates1_CSIT_beam.append(r)
